# Исследование надежности заемщиков

   
**Краткое описание:** Заказчик — кредитный отдел банка.  Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
    
**Цель проекта:** Определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
    
**Описание данных:** Входные данные от банка, подлежащие анализу, — статистика о платёжеспособности клиентов. Таблица с данными состоит из следующих колонок:

* `'children'` — количество детей в семье.
* `'days_employed'` — общий трудовой стаж в днях.
* `'dob_years'` — возраст клиента в годах.
* `'education'` — уровень образования клиента.
* `'education_id'` — идентификатор уровня образования.
* `'family_status'` — семейное положение.
* `'family_status_id'` — идентификатор семейного положения.
* `'gender'` — пол клиента.
* `'income_type'` — тип занятости.
* `'debt'` — имел ли задолженность по возврату кредитов.
* `'total_income'` — ежемесячный доход.
* `'purpose'` — цель получения кредита.
    
**План работы:**
1. выгрузить данные, осуществить первичный обзор;
2. определить, есть ли пропуски в данных; заполнить пропуски или удалить строки с пропусками;
3. проверить данные на аномалии, исправить их;
4. проверить данные на наличие явных и неявных дубликатов, исправить данные;
5. изменение типов данных;
6. формирование новых датафреймов, декомпозиция основного датафрейма;
7. анализ данных в соответствии с поставленной целью проекта;
8. сформировать вывод, полученный из результатов анализа.

### Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

<b>Проблемы, выявенные после первичного обзора данных:</b>

1. В колонке `'days_employed'` должен быть указан трудовой стаж в днях. Однако в ней есть отрицательные значения, а сами значения кажутся слишком большими, что нереалистично.


2. В колонках `'total_income'` и `'days_employed'` есть пропуски.


3. В колонке `'education'` строчные буквы сочетаются с прописными.


### Заполнение пропусков

#### Расчет количества пропущенных значений в таблице.

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в колонках `'total_income'` и `'days_employed'` совпадает и равняется `2174`. Эти пропуски могут быть взаимосвязаны, поскольку неработающие люди не имеют ежемесячного дохода.

#### Вывод строк с пропущенными значениями, отвечающих двум условиям.

In [6]:
df.loc[
       (df['days_employed'].isna() == True) &
       (df['total_income'].isna() == True)
       ]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Всего получилось `2174` строки. Значит, теория о том, что люди без стажа работы не имеют дохода, подтвердилась.
Следовательно, пропуски не случайны, а имеют реалистичную причину.

#### Определение доли пропущенных значений

Определим долю пропусков в процентах.

In [7]:
(df.isna().mean()*100).round(1)

children             0.0
days_employed       10.1
dob_years            0.0
education            0.0
education_id         0.0
family_status        0.0
family_status_id     0.0
gender               0.0
income_type          0.0
debt                 0.0
total_income        10.1
purpose              0.0
dtype: float64

Получается, что каждая десятая строка имеет пропуски, что довольно много. Поскольку нам необходимо проанализировать семейное положение и количество детей клиента на факт погашения кредита в срок, то удалять строки с пропусками данных о стаже работы и доходе не целесообразно. Эти строки имеют необходимые для анализа данные, поэтому их надо заполнить.

#### Проверка и исправление данных о трудовом стаже и ежемесячном доходе.

Прежде, чем заполнить пропуски, необходимо исправить ошибки в данных.

 Из пояснений к данным видно, что в колонке `'days_employed'` отображен общий трудовой стаж в днях. Однако в данной колонке есть отрицательные значения. Это может быть вызвано ошибкой в записи данных. Необходимо отрицательные значения сделать положительными.

In [8]:
df['days_employed'] = abs(df['days_employed'])
df['days_employed'].min() # проверяем наличие / отсутсвие отрицательных значений в колонке

24.14163324048118

Отрицательных значений нет.

Также в колонке `'days_employed'` трудовой стаж должен быть отображен в днях, однако имеющиеся значения кажутся нереалистичными. Попробуем это проверить, определив максимальное значение в столбце и разделив его на 365 дней. Отображение количества лет будет наглядным.

In [9]:
int(df['days_employed'].max() / 365)

1100

Определенно, люди не живут 1100 лет =)

Значит, данные представлены не в днях, а, вероятнее, в часах. Попробуем это проверить, разделив максимальное значение столбца сначала на 24 часа, а затем  - на 365 дней.

In [10]:
int(df['days_employed'].max() / 24 / 365)

45

Получается 45 лет. Такой стаж работы вполне возможен. Значит, данные в колонке `'days_employed'` отображают общий трудовой стаж не в днях, а в часах. В данном случае будет достаточно переименовать наименование колонки, чтобы оно отображало данные, которые в ней находятся.

In [11]:
df = df.rename(columns={'days_employed' : 'hours_employed'})
df.head() # проверяем, как теперь выглядит таблица

,children,hours_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверим, есть ли отрицательные значения ежемесячного дохода, определив минимальное значение в колонке `total_income`.

In [12]:
df['total_income'].min()

20667.26379327158

Таких значений нет, значит, данные не требуют исправлений.

#### Заполнение пропусков

Обе колонки с пропусками имеют количественные значения. В данном случае пропуски лучше заполнять медианным значением, чем средним. Продемонстрируем на примере колонки `'hours_employed'`. Вызовим ее описание.

In [13]:
df['hours_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: hours_employed, dtype: float64

Из описания видно, что минимальное и максимальное значения в колонке сильно различаются. Это влияет на среднее значение (все помнят про капусту у бедняков, мясо у богатых, а среднее - это голубцы). Медианное же значение отображет значение, которое находится посередине и более точно выражает вероятные данные, которые пропущены.

Заполняем пропущенные значения трудового стажа и ежемесячного дохода медианным значением.

In [14]:
df['hours_employed'] = df['hours_employed'].fillna(df['hours_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df.isna().sum() # проверяем, все ли пропуски заполнены.

children            0
hours_employed      0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления.

Чтобы проверить, есть ли проблемы с данными, можно использовать метод `value_counts()`. Он покажет количество уникальных значений в столбце.

#### Проверка и исправление данных о количестве детей.

In [15]:
df['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Значение `-1` может быть ошибкой в записи данных. Сделаем отрицательных значения положительными.

In [16]:
df['children'] = abs(df['children'])

Вероятность того, что у одного человека есть 20 детей, крайне мала, а у 76 - и подавно. =)

Это тоже может быть результатом влияния человеческого фактора. Заменим все значения `20` на `2`. Так малозначительная группа присоединится к крупной группе данных.

In [17]:
df.loc[(df['children'] == 20), 'children'] = 2

# проверяем количество уникальных значений в столбце после внесения изменений.
df['children'].value_counts().sort_index()  

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Проверка и исправление данных о возрасте клиентов

In [18]:
df['dob_years'].value_counts().sort_index().head()

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64

Оказывается, что новорожденные не против взять на себя такое бремя, как кредит =)

Значение `0` - это явно ошибка в данных. Это может означать отсутвие данных о возрасте или неправильную запись данных. Чтобы не удалять строки, заменим эти значения медианным. Тип данных в  колонке `dob_years` изменится на `float24`, вернем тип `int24`.

In [19]:
df.loc[((df['dob_years'] == 0) | (df['dob_years'] == 2)), 'dob_years'] = (
    df['dob_years'].median())

df['dob_years'] = df['dob_years'].astype('int')

# проверяем, все ли получилось
df['dob_years'].value_counts().sort_index().head()

19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64

#### Проверка других столбцов с численными значениями

Проверим другие столбцы с численными значениями.

In [20]:
df['education_id'].value_counts().sort_index()

0     5260
1    15233
2      744
3      282
4        6
Name: education_id, dtype: int64

In [21]:
df['family_status_id'].value_counts().sort_index()

0    12380
1     4177
2      960
3     1195
4     2813
Name: family_status_id, dtype: int64

In [22]:
df['debt'].value_counts().sort_index()

0    19784
1     1741
Name: debt, dtype: int64

В столбцах `'education_id'`, `'family_status_id'` и `'debt'` аномалий не обнаружено.

### Удаление дубликатов.

#### Удаление дубликатов в колонке `'education'`

После первичного обзора данных было обнаружено, что в колонке `'education'` строчные буквы сочетаются с прописными. Проверим это, вызвав метод `value_counts()`.

In [23]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Предположение подтвердилось: в этой колонке есть дубликаты. Дубликаты могут появиться из-за того, что данные вносят разные люди. Кто-то пишет с большой буквы, кто-то капсом. Для начала приведем все символы в строках к нижнему регистру вызовом метода `str.lower()`. 

In [24]:
df['education'] = df['education'].str.lower()

# проверяем количество уникальных значений в столбце после внесения изменений
df['education'].value_counts() 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь видно, что есть 5 категорий данных об образовании. Если сопоставить колонки `'education'` и `'education_id'`, то видно, что 
* `0` - `высшее`
* `1` - `среднее`
* `2` - `неоконченное высшее`
* `3` - `начальное`
* `4` - `ученая степень`

#### Проверка на наличие дубликатов в колонке `'family_status'`

In [25]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Дубликатов здесь нет, но используются разные регистры написания. Приведем все символы в строках к нижнему регистру вызовом метода `str.lower()`

In [26]:
df['family_status'] = df['family_status'].str.lower()

# проверяем, как выглядят уникальные значения в столбце после внесения изменений
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Также видно, что есть 5 категорий данных о семейном положении. Если сопоставить колонки `'family_status'` и `'family_status_id'`, то видно, что 

* `0` - `женат / замужем`
* `1` - `гражданский брак`
* `2` - `вдовец / вдова`
* `3` - `в разводе`
* `4` - `не женат / не замужем`

#### Проверка на наличие дубликатов в колонке `'income_type'`

In [27]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

В столбце `'income_type'` дубликатов нет, но есть малозначительные группы: `предприниматель`, `безработный`, `в декрете` и `студент`. Эти строки можно было бы удалить или распределить малые группы по большим, но посольку нам необходимо проанализировать влияние семейного положения и количества детей клиента на факт погашения кредита в срок, это можно не делать. В анализе будут использованы числовые данные о доходе, а не тип доходов.

#### Проверка на наличие дубликатов в колонке `'gender'`

In [28]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце 'gender' дубликатов нет, но есть одно значение '`XNA`'. Возможно, понадобится исследования, связанные с полом, а это значение может повлиять на результаты анализа. Удалим эту строку, перезаписав dataframe без строки со значением '`XNA`'.

In [29]:
df = df.loc[df['gender'] != 'XNA']

# проверяем, как выглядят уникальные значения в столбце после внесения изменений.
df['gender'].value_counts() 

F    14236
M     7288
Name: gender, dtype: int64

#### Проверка на наличие явных дубликатов

Посмотрим количество явных дубликов в таблице.

In [30]:
df.duplicated().sum()

72

Выведим на экран первые 10 дубликатов.

In [31]:
df[df.duplicated()].head(10)

,children,hours_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194.220567,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017.937533,покупка жилья для семьи
3290,0,2194.220567,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,сыграть свадьбу
4182,1,2194.220567,34,высшее,0,гражданский брак,1,F,сотрудник,0,145017.937533,свадьба
4851,0,2194.220567,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,свадьба
5557,0,2194.220567,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,сыграть свадьбу
6312,0,2194.220567,30,среднее,1,женат / замужем,0,M,сотрудник,0,145017.937533,строительство жилой недвижимости
7808,0,2194.220567,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,на проведение свадьбы
7921,0,2194.220567,64,высшее,0,гражданский брак,1,F,пенсионер,0,145017.937533,на проведение свадьбы
7938,0,2194.220567,71,среднее,1,гражданский брак,1,F,пенсионер,0,145017.937533,на проведение свадьбы
8583,0,2194.220567,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,145017.937533,дополнительное образование


Посчитаем долю дубликатов в датафрейме в процентах.

In [32]:
(df.duplicated().sum() / df.count() * 100).round(1)

children            0.3
hours_employed      0.3
dob_years           0.3
education           0.3
education_id        0.3
family_status       0.3
family_status_id    0.3
gender              0.3
income_type         0.3
debt                0.3
total_income        0.3
purpose             0.3
dtype: float64

Доля дубликатов составляет `0.3%`. Это не так много, поэтому явные дубликаты можно удалить. Перезапишем датафрейм без дубликатов.

In [33]:
df = df.drop_duplicates().reset_index(drop=True)

#проверяем, остались ли дубликаты в таблице
df.duplicated().sum()

0

### Изменение типов данных.

Для упрощения анализа и большей наглядности стоит поменять тип данных в колонках `hours_employed` и `total_income`. Часы - маленькая единица значения, будет достаточно выражения в целых числах. Так же и с доходом: копейки не так важны.

Также заменим тип колонки `'debt'` на `bool`. Теперь вместо `0` и `1` в колонке будет отображаться `False` и `True` соответственно.

In [34]:
df['hours_employed'] = df['hours_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['debt'] = df['debt'].astype('bool')

df.info() # проверяем, поменялся ли тип данных.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21452 non-null  int64 
 1   hours_employed    21452 non-null  int64 
 2   dob_years         21452 non-null  int64 
 3   education         21452 non-null  object
 4   education_id      21452 non-null  int64 
 5   family_status     21452 non-null  object
 6   family_status_id  21452 non-null  int64 
 7   gender            21452 non-null  object
 8   income_type       21452 non-null  object
 9   debt              21452 non-null  bool  
 10  total_income      21452 non-null  int64 
 11  purpose           21452 non-null  object
dtypes: bool(1), int64(6), object(5)
memory usage: 1.8+ MB


Посмотрим, как теперь выглядят данные в таблице.

In [35]:
df.head()

,children,hours_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу


### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Данные об образовании и семейном статусе выражены не только в виде наименований, но и в виде индентификаторов. Создадим два датафрейма-словаря для этих данных, чтобы облегчить основной датафрейм.

In [36]:
df_education = (
    df[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
)

df_education # проверяем содержание нового датафрейма.

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [37]:
df_family_status = (
    df[['family_status', 'family_status_id']]
    .drop_duplicates()
    .reset_index(drop=True)
)

df_family_status # проверяем содержание нового датафрейма.

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим столбцы `'family_status'` и `'education'` из основного датафрейма, чтобы его облегчить.

In [38]:
df = df.drop (columns=['family_status', 'education'])

df.head() # проверяем содержание нового датафрейма.

,children,hours_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,False,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,False,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,False,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,False,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,False,158616,сыграть свадьбу


### Категоризация дохода.

Для упрощения анализа разделим доход на категории:
* `до 30000 руб.` — `E`;
* `30001–50000 руб.` — `D`;
* `50001–200000 руб.` — `C`;
* `200001–1000000 руб.` — `B`;
* `1000001 руб. и выше` — `A`.

In [39]:
def income_category(income):
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    elif 1000001 <= income:
        return 'A'
    
df['total_income_category'] = df['total_income'].apply(income_category)

# проверяем, как выглядят уникальные значения в новом столбце.
df['total_income_category'].value_counts().sort_index() 

A       25
B     5040
C    16015
D      350
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита.

In [40]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В колонке `'purpose'` большое количество неявных дубликатов: цели имеют один смысл, а описаны по-разному. Попробуем это исправить, заменив наименование целей с ключевыми словами на единое обозначение.

In [41]:
def purpose_category(word):
    if 'авто' in word:
        return 'операции с автомобилем'
    elif 'свадьб' in word:
        return 'проведение свадьбы'
    elif 'недвиж' in word or 'жиль' in word:
        return 'операции с недвижимостью'
    elif 'образов' in word:
        return 'получение образования'
        
df['purpose_category'] = df['purpose'].apply(purpose_category)

 # проверяем, как выглядят уникальные значения в новом столбце.
df['purpose_category'].value_counts()

операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

### Категоризация количества детей.

При проверке данных о количестве детей было заметно, что количество значений равных `4` и `5` очень мало. Их следует добавить в группу со значением `3`. Создадим новый столбец с категориями количества детей и добавим ее в основной датафрейм.

In [42]:
# выведим количество уникальных значений столбца 'children'
df['children'].value_counts().sort_index() 

0    14089
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [43]:
def children_category(child):
    if child == 0:
        return 'нет детей'
    elif child == 1:
        return 'один ребенок'
    elif child == 2:
        return 'двое детей'
    elif child == 3 or child == 4 or child == 5:
        return 'трое и более детей'
        
df['children_category'] = df['children'].apply(children_category)

# проверяем, как выглядят уникальные значения в новом столбце
df['children_category'].value_counts()

нет детей             14089
один ребенок           4855
двое детей             2128
трое и более детей      380
Name: children_category, dtype: int64

Посмотрим, как выглядит новый датафрейм, готовый к дальнейшему анализу.

In [44]:
# изменяем порядок столбцов
df = df[['children', 'children_category', 'hours_employed', 'dob_years', 'education_id',
         'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
         'total_income_category', 'purpose', 'purpose_category']]

df.head() # проверяем содержание нового датафрейма

,children,children_category,hours_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose,purpose_category
0,1,один ребенок,8437,42,0,0,F,сотрудник,False,253875,B,покупка жилья,операции с недвижимостью
1,1,один ребенок,4024,36,1,0,F,сотрудник,False,112080,C,приобретение автомобиля,операции с автомобилем
2,0,нет детей,5623,33,1,0,M,сотрудник,False,145885,C,покупка жилья,операции с недвижимостью
3,3,трое и более детей,4124,32,1,0,M,сотрудник,False,267628,B,дополнительное образование,получение образования
4,0,нет детей,340266,53,1,1,F,пенсионер,False,158616,C,сыграть свадьбу,проведение свадьбы


### Ответы на вопросы.

Прежде, чем отвечать на поставленные вопросы, посчитаем процент невозвращенных кредитов в зависимости от общего количества кредитов. Так мы определим среднюю статистику, с которой мы будем сравнивнивать статистику по отдельным категориям.

In [45]:
percent_debt_true = (
    (df[df['debt'] == True].count() / df.count() * 100).mean().round(1)
)

print('Процент невозвращенных кредитов:', percent_debt_true)

Процент невозвращенных кредитов: 8.1


##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создадим сводную таблицу, которая покажет количество клиентов в зависимости от наличия детей, их количества и наличия задолженности по кредиту.

In [46]:
df_children_debt = df.pivot_table(
    index='children_category',
    columns='debt',
    values='dob_years',
    aggfunc='count'
    )

df_children_debt

debt,False,True
children_category,,
двое детей,1926,202
нет детей,13026,1063
один ребенок,4410,445
трое и более детей,349,31


Посчитаем процент невозвращенных в срок кредитов для каждой категории клиентов с детьми. Запишем результаты в новой колонке `'percent'`.

In [48]:
df_children_debt['percent'] = (
    (df_children_debt[1] / (df_children_debt[1]
                           + df_children_debt[0]) * 100).round(1)
                           )

#сортировка данных по количеству возвращенных кредитов
df_children_debt.sort_values(by=1, ascending=False)

debt,False,True,percent
children_category,,,
нет детей,13026,1063,7.5
один ребенок,4410,445,9.2
двое детей,1926,202,9.5
трое и более детей,349,31,8.2


In [49]:
#сортировка данных по проценту невозвращенных кредитов
df_children_debt.sort_values(by='percent', ascending=False)

debt,False,True,percent
children_category,,,
двое детей,1926,202,9.5
один ребенок,4410,445,9.2
трое и более детей,349,31,8.2
нет детей,13026,1063,7.5


Расчитаем средний процент невозвращенных кредитов среди клиентов, у которых есть дети.

In [50]:
df_children_debt['percent'].loc[['один ребенок', 'двое детей', 'трое и более детей']].mean().round(1)

9.0

##### Вывод 1:

Из результатов видно, что наибольшее количество людей, которые обращаются за кредитами, не имеют детей. Количество кредитов уменьшается в зависимости от увеличения количества детей.

При этом наименьший процент невозвращенных кредитов именно у категории людей, не имеющих детей - `7.5%`, что меньше средних `8.1%` по всем данным.

Процент невозвращенных кредитов у клиентов, у которых `один ребенок` и `двое детей`, увеличивается до `9.2%` и `9.5%` соответственно, что больше среднего.

У тех, у кого `трое и более детей`, процент неоплат снижается до `8.2%`.

Если сравнивать результаты в разрезе "нет детей - есть дети", то наличие детей влияет на возврат кредита.
* Клиенты, у которых нет детей, не возвращают `7.5%` кредитов, что меньше среднего процента невозврата, равного `8.1%`.

* Клиенты, у которых есть дети, не возвращают `9.0%` кредитов, что больше среднего процента невозврата.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу, которая покажет количество клиентов в зависимости от семейного положения и наличия задолженности по кредиту.

In [51]:
df_family_status_debt = df.pivot_table(
    index='family_status_id',
    columns='debt',
    values='dob_years',
    aggfunc='count'
    )

df_family_status_debt

debt,False,True
family_status_id,,
0,11408,931
1,3761,388
2,896,63
3,1110,85
4,2536,274


Посчитаем процент невозвращенных в срок кредитов для каждой категории клиентов. Запишем результаты в новой колонке `'percent'`.

In [53]:
df_family_status_debt['percent'] = (
    (df_family_status_debt[1] / (df_family_status_debt[1]
                                + df_family_status_debt[0]) * 100).round(1)
                                )

#сортировка данных по количеству возвращенных кредитов.
df_family_status_debt.sort_values(by=1, ascending=False) 

debt,False,True,percent
family_status_id,,,
0,11408,931,7.5
1,3761,388,9.4
4,2536,274,9.8
3,1110,85,7.1
2,896,63,6.6


In [54]:
#сортировка данных по проценту невозвращенных кредитов
df_family_status_debt.sort_values(by='percent', ascending=False) 

debt,False,True,percent
family_status_id,,,
4,2536,274,9.8
1,3761,388,9.4
0,11408,931,7.5
3,1110,85,7.1
2,896,63,6.6


Напомним соответствие id семейному статусу.

In [55]:
df_family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Расчитаем средний процент невозвращенных кредитов среди клиентов, которые состоят в браке или состояли.

In [56]:
df_family_status_debt['percent'].loc[[0, 2, 3]].mean().round(1)

7.1

Расчитаем средний процент невозвращенных кредитов среди клиентов, которые в браке не состоят и не состояли.

In [57]:
df_family_status_debt['percent'].loc[[1, 4]].mean().round(1)

9.6

##### Вывод 2:

Из результатов видно, что наибольшее количество людей, которые обращаются за кредитами, состоят в официальном браке, т.е. относятся к категории `женат / замужем`. Клиенты из других категорий обращаются за кредитами намного реже.

Клиенты из категорий `вдовец / вдова`, `в разводе`, `женат / замужем` чаще возвращают кредиты срок. Процент невозвращенных кредитов среди этих категорий ниже средних `8.1%` и составляет `6.6%`, `7.1%`, `7.5%` соответственно.

Клиенты, находящиеся в гражданском браке или, которые в браке никогда не состояли, возвращают кредиты значительно хуже. Процент невозвращенных кредитов для категорий `гражданский брак` и `не женат / не замужем` составляет `9.4%` и `9.8%` соответственно.

Из вышеизложенного делаем вывод, что семейное положение влияет на возможность вернуть кредитные средства банку: клиенты, которые состоят в официальном браке или когда-либо состояли, погашают долги в срок чаще, чем клиенты, которые в браке не состоят и не состояли.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу, которая покажет количество клиентов в зависимости от категории дохода и наличия задолженности по кредиту.

In [58]:
df_total_income_debt = df.pivot_table(
    index='total_income_category',
    columns='debt',
     values='dob_years',
     aggfunc='count'
     )

df_total_income_debt

debt,False,True
total_income_category,,
A,23,2
B,4684,356
C,14655,1360
D,329,21
E,20,2


Посчитаем процент невозвращенных в срок кредитов для каждой категории клиентов. Запишем результаты в новой колонке `'percent'`.

In [59]:
df_total_income_debt['percent'] = (
    df_total_income_debt[1] / (df_total_income_debt[1]
                               + df_total_income_debt[0]) * 100
)

#сортировка данных по количеству возвращенных кредитов.
df_total_income_debt.sort_values(by=1, ascending=False).round(1)

debt,False,True,percent
total_income_category,,,
C,14655,1360,8.5
B,4684,356,7.1
D,329,21,6.0
A,23,2,8.0
E,20,2,9.1


In [60]:
#сортировка данных по проценту невозвращенных кредитов
df_total_income_debt.sort_values(by='percent', ascending=False).round(1)

debt,False,True,percent
total_income_category,,,
E,20,2,9.1
C,14655,1360,8.5
A,23,2,8.0
B,4684,356,7.1
D,329,21,6.0


Напомним, что скрывается за буквами, обозначающими категорию дохода.

* `до 30000 руб.` — `E`;
* `30001–50000 руб.` — `D`;
* `50001–200000 руб.` — `C`;
* `200001–1000000 руб.` — `B`;
* `1000001 руб. и более` — `A`.

##### Вывод 3:

Из сводной таблицы видно, что за кредитами чаще всего обращаются люди с доходом `50001–200000 руб.`, далее - люди с доходом  `200001–1000000 руб.`. Люди обладающие меньшим или большим уровнем дохода обращаются за получением кредитных средств намного реже. Это также может быть связано со средним уровнем дохода среди населения.

Удивительный для меня инсайт - это то, что высокий уровень дохоада не гарантирует возврат кредита в срок. Наиболее ответственная категория - категория `D` с доходом `30001–50000 руб.`; процент невозврата для этой категории стоставляет `6.0%`, что меньше средних `8.1%`. Еще одна категория с более низким процентом невозвращенных кредитов - категория `B` с доходом `200001–1000000 руб.` - `7.1%` непогашенных долгов.

У самой многочисленной категории - категории `C` с доходом `50001–200000 руб.` - процент невозвратов немного превышает средний показатель и составляет `8.5%`. А самая неплатежеспособная группа - группа `E` с доходом до `30000 руб.`; процент невозвратов составляет `9.1%`.

Основные риски невозврата кредитных средств в срок связаны с людьми, имеющий низкий уровень доходов до 30000 руб. / мес. Остальные группы показывают процент невозврата ниже среднего или близко к среднему, поэтому можно сказать, что уровень дохода - не основной показатель, который влияет на возврат долга.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу, которая покажет количество клиентов в зависимости от цели кредита и наличия задолженности по кредиту.

In [61]:
df_purpose_debt = df.pivot_table(
    index='purpose_category',
    columns='debt',
    values='dob_years',
    aggfunc='count'
    )

df_purpose_debt

debt,False,True
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,10028,782
получение образования,3643,370
проведение свадьбы,2137,186


Посчитаем процент невозвращенных в срок кредитов для каждой категории клиентов. Запишем результаты в новой колонке `'percent'`.

In [64]:
df_purpose_debt['percent'] = (
    (df_purpose_debt[1] / (df_purpose_debt[1]
                          + df_purpose_debt[0]) * 100).round(1)
                          )

#сортировка данных по количеству возвращенных кредитов.
df_purpose_debt.sort_values(by=1, ascending=False) 

debt,False,True,percent
purpose_category,,,
операции с недвижимостью,10028,782,7.2
операции с автомобилем,3903,403,9.4
получение образования,3643,370,9.2
проведение свадьбы,2137,186,8.0


In [65]:
#сортировка данных по проценту невозвращенных кредитов.
df_purpose_debt.sort_values(by='percent', ascending=False).round(1)

debt,False,True,percent
purpose_category,,,
операции с автомобилем,3903,403,9.4
получение образования,3643,370,9.2
проведение свадьбы,2137,186,8.0
операции с недвижимостью,10028,782,7.2


##### Вывод 4:

Основная группа людей берут кредит для того, чтобы осуществить операции с недвижимостью. Эта же группа чаще других возвращает кредит в срок: процент невозвратов составляет `7.2%`, что ниже средних `8.1%`.

Реже всего погашают долги те, кто обратился за кредитом для получения образования и осуществления операций с автомобилем. Процент невозврата долга для этих категорий составляет `9.2%` и `9.4%` соответственно.

При анализе данной категории данных четко прослеживается влияние цели кредита на возможность его возврата в срок: кредиты, за которыми обращались для проведения свадьбы или операций с недвижимостью возвращают вовремя чаще, чем кредиты для получения образования или операций с автомобилем. 

## Общий вывод:

В ходе исследования необходимо было проанализировать влияние различных показателей на возврат кредита в срок.
Было обнаружено следующее:

* Наличие детей влияет на возможность вернуть кредит в срок: клиенты с детьми возвращают кредит в срок на 20% процентов (или 1.5 процентных пункта) реже, чем те, у кого детей нет. Наблюдается увеличение количества невозвращенных кредитов в зависимости от количества детей. При этом те, у кого трое и более детей, не имеют задолженности чаще, чем те, у кого один ребенок или двое детей.

* Семейное положение влияет на возможность вернуть кредит в срок: клиенты, которые состоят в официальном браке или когда-либо состояли, погашают долги в срок на 35% (или 2.5 процентных пункта) чаще, чем клиенты, которые в браке не состоят и не состояли.

* Уровень дохода не сильно влияет на возможность вернуть кредит в срок: разные категории показывают процент невозврата ниже среднего или близко к среднему. При этом основные риски невозврата кредитных средств в срок связаны с людьми, имеющий низкий уровень доходов до 30000 руб. / мес, т.к. процент невозврата на 12% (или 1 процентный пункт) больше среднего по всем показателям.

* Цель кредита влияет на возможность вернуть кредит в срок: кредиты, за которыми обращались для проведения свадьбы или операций с недвижимостью возвращают вовремя чаще, чем кредиты для получения образования или операций с автомобилем. Кредиты для получения образования или операций с автомобилем имеют процент невозврата кредитных средств в срок на 13.5% и 16% (или 1.1 и 1.3 процентных пункта) больше среднего по всем показателям соответвенно. Кредиты для проведения свадьбы или операций с недвижимостью имеют процент невозврата близкий к среднему по всем показателям соответвенно.